<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/nlp802-paradetox-1Token-Split-MASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install transformers[torch]
!pip install bert-score
!pip install huggingface_hub

In [ ]:
!huggingface-cli login

In [83]:
import torch
from bert_score import score
from datasets import load_dataset

paradetox_1token_train = load_dataset("HamdanXI/paradetox-1Token-Split", split='train')
paradetox_1token_valid = load_dataset("HamdanXI/paradetox-1Token-Split", split='validation')
paradetox_1token_test = load_dataset("HamdanXI/paradetox-1Token-Split", split='test')
# paradetox = load_dataset("HamdanXI/paradetox-preprocess-maskedComments-without-INSERT-without-punctationComparision")

In [84]:
# paradetox = paradetox['train'].remove_columns(['edit_ops', 'en_toxic_comment'])
paradetox_1token_train = paradetox_1token_train.remove_columns(['edit_ops', 'en_toxic_comment'])
paradetox_1token_valid = paradetox_1token_valid.remove_columns(['edit_ops', 'en_toxic_comment'])
paradetox_1token_test = paradetox_1token_test.remove_columns(['edit_ops', 'en_toxic_comment'])

In [85]:
import pandas as pd

# df_paradetox = pd.DataFrame(paradetox)
df_paradetox_1token_train = pd.DataFrame(paradetox_1token_train)
df_paradetox_1token_valid = pd.DataFrame(paradetox_1token_valid)
df_paradetox_1token_test = pd.DataFrame(paradetox_1token_test)

# df_paradetox['masked_comment'] = df_paradetox['masked_comment'].str.replace(r'\[REPLACE\]|\[DELETE\]', '[MASK]', regex=True)
df_paradetox_1token_train['masked_comment'] = df_paradetox_1token_train['masked_comment'].str.replace(r'\[REPLACE\]|\[DELETE\]', '[MASK]', regex=True)
df_paradetox_1token_valid['masked_comment'] = df_paradetox_1token_valid['masked_comment'].str.replace(r'\[REPLACE\]|\[DELETE\]', '[MASK]', regex=True)
df_paradetox_1token_test['masked_comment'] = df_paradetox_1token_test['masked_comment'].str.replace(r'\[REPLACE\]|\[DELETE\]', '[MASK]', regex=True)

In [44]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

# Load tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [86]:
train_max_token_length_input = max(len(tokenizer.encode(item)) for item in df_paradetox_1token_train['en_neutral_comment'])
train_max_token_length_label = max(len(tokenizer.encode(item)) for item in df_paradetox_1token_train['masked_comment'])

valid_max_token_length_input = max(len(tokenizer.encode(item)) for item in df_paradetox_1token_valid['en_neutral_comment'])
valid_max_token_length_label = max(len(tokenizer.encode(item)) for item in df_paradetox_1token_valid['masked_comment'])

test_max_token_length_input = max(len(tokenizer.encode(item)) for item in df_paradetox_1token_test['en_neutral_comment'])
test_max_token_length_label = max(len(tokenizer.encode(item)) for item in df_paradetox_1token_test['masked_comment'])

if train_max_token_length_input > train_max_token_length_label:
    highest_length_train = train_max_token_length_input
else:
    highest_length_train = train_max_token_length_label

if valid_max_token_length_input > valid_max_token_length_label:
    highest_length_valid = valid_max_token_length_input
else:
    highest_length_valid = valid_max_token_length_label

if test_max_token_length_input > test_max_token_length_label:
    highest_length_test = test_max_token_length_input
else:
    highest_length_test = test_max_token_length_label

if highest_length_train > highest_length_valid:
    highest_length = highest_length_train
else:
    highest_length = highest_length_valid

if highest_length_test > highest_length:
    highest_length = highest_length_test

print(highest_length)

32


In [87]:
from datasets import Dataset

# Assuming your DataFrame is named 'df'
paradetox_1token_train = Dataset.from_pandas(df_paradetox_1token_train)
paradetox_1token_valid = Dataset.from_pandas(df_paradetox_1token_valid)
paradetox_1token_test = Dataset.from_pandas(df_paradetox_1token_test)

paradetox_1token_train = paradetox_1token_train.rename_column("masked_comment", "input")
paradetox_1token_train = paradetox_1token_train.rename_column("en_neutral_comment", "labels")

paradetox_1token_valid = paradetox_1token_valid.rename_column("masked_comment", "input")
paradetox_1token_valid = paradetox_1token_valid.rename_column("en_neutral_comment", "labels")

paradetox_1token_test = paradetox_1token_test.rename_column("masked_comment", "input")
paradetox_1token_test = paradetox_1token_test.rename_column("en_neutral_comment", "labels")

In [90]:
paradetox_1token_train

Dataset({
    features: ['labels', 'input'],
    num_rows: 3784
})

In [89]:
paradetox_1token_valid

Dataset({
    features: ['labels', 'input'],
    num_rows: 811
})

In [88]:
paradetox_1token_test

Dataset({
    features: ['labels', 'input'],
    num_rows: 811
})

In [92]:
from datasets import DatasetDict

merged_dataset = DatasetDict({
    'train': paradetox_1token_train,
    'validation': paradetox_1token_valid,
    'test': paradetox_1token_test
})

In [93]:
merged_dataset.push_to_hub("paradetox-1Token-Split-MASK")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [74]:
def tokenize_function(example):
    source = example["input"]
    target = example["labels"]

    # Tokenizing source and target without returning tensors and without padding
    tokenized_source = tokenizer(source, truncation=True, max_length=highest_length, return_tensors="pt")
    tokenized_target = tokenizer(target, truncation=True, max_length=highest_length, return_tensors="pt")

    return {
        "input_ids": tokenized_source["input_ids"][0],
        "attention_mask": tokenized_source["attention_mask"][0],
        "labels": tokenized_target["input_ids"][0]
    }

tokenized_paradetox_1token_train_datasets = paradetox_1token_train.map(tokenize_function)
tokenized_paradetox_1token_valid_datasets = paradetox_1token_valid.map(tokenize_function)

Map:   0%|          | 0/3784 [00:00<?, ? examples/s]

Map:   0%|          | 0/811 [00:00<?, ? examples/s]

In [81]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(output_dir="bart-base-paradetox-1Token-split-masked-v2", push_to_hub=True)
training_args = training_args.set_save(steps=10000)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_paradetox_1token_train_datasets,
    eval_dataset=tokenized_paradetox_1token_valid_datasets,
    tokenizer=tokenizer,
)

In [82]:
trainer.train()

ValueError: ignored

In [30]:
# Function to replace mask tokens with predictions
def predict_masked_sentences(sentences, tokenizer, model):
    predictions = []
    for sentence in sentences:
        # Tokenize input
        input = tokenizer.encode(sentence, return_tensors="pt")

        # Get the mask token index
        mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

        # Predict all tokens
        with torch.no_grad():
            output = model(input)
            prediction = output.logits

        # For each mask token, get the top predicted token
        for mask_index in mask_token_index:
            predicted_token_id = prediction[0, mask_index].argmax(axis=-1)
            predicted_token = tokenizer.decode(predicted_token_id)
            sentence = sentence.replace(tokenizer.mask_token, predicted_token, 1)

        predictions.append(sentence)

    return predictions

In [32]:
# Generate predictions
predicted_sentences = predict_masked_sentences(df_paradetox_1token['masked_comment'], tokenizer, model)
# print(predicted_sentences)

In [36]:
predicted_sentences[1]

'[MASK] let me in mysch then bro.'

In [ ]:
# Function to predict the masked word
def predict_masked_word(masked_text):
    inputs = tokenizer(masked_text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    masked_index = (inputs.input_ids[0] == tokenizer.mask_token_id).nonzero(as_tuple=True)[0]
    predicted_token_id = logits[0, masked_index, :].argmax(axis=-1)
    predicted_token = tokenizer.decode(predicted_token_id)
    return masked_text.replace(tokenizer.mask_token, predicted_token)

# Generate predictions and compute BERTScore
bert_scores = []
for i in range(len(dataset['test'])):
    toxic_comment = dataset['test'][i]['en_toxic_comment']
    neutral_comment = dataset['test'][i]['en_neutral_comment']
    masked_comment = dataset['test'][i]['masked_comment']

    # Generate prediction
    prediction = predict_masked_word(masked_comment)

    # Compute BERTScore
    # P, R, F1 = score([prediction], [neutral_comment], lang="en", verbose=True)
    # bert_scores.append(F1.mean().item())

# Calculate the average BERTScore
# average_bert_score = sum(bert_scores) / len(bert_scores)
# print(f"Average BERTScore: {average_bert_score}")

In [ ]:
print(f"Average BERTScore: {average_bert_score}")

Average BERTScore: 0.9379353561471923


In [5]:
prediction

'on the grounds that all the parties are [REPLACE]'

In [ ]:
!pip install git+https://github.com/google-research/bleurt.git

In [ ]:
from datasets import load_metric

bleurt_metric = load_metric('bleurt')

In [9]:
score_results = bleurt_metric.compute(predictions=prediction, references=dataset['test']['en_neutral_comment'])

scores = score_results['scores']

average_score = sum(scores) / len(scores) if scores else 0
print(f"Average BLEURT Score: {average_score}")

ValueError: ignored